<h1>Making a binary bubble map</h1>
<p>In this example, we will look at making two similar bubble maps that bypass the normal data loading and processing steps and instead package all of the data as a binary file.</p>

In [ ]:
import pandas as pd
df = pd.read_csv('../data/examples/DataRecords.csv')
df.head()

In [ ]:
len(df), len(df['Date'].unique())

<h1>Rationale</h1>
<p>We have 10000 plus observations that have 4383 unique dates associated. These dates are ad hoc and not regularly spaced so formatting this data into a standard EarthTime CSV format woukld result in an excessively large file. What we want to do is preprocess our data in some way so that we can still use the bubble map shaders.</p>
<p>
    We can determine our required data format by looking at the vertext shader code.<br/>
    <strong>WebGLVectorTile2Shaders.bubbleMapWithPackedColorVertexShader</strong>
    <code>
attribute vec4 a_Centroid;
attribute float a_Epoch1;
attribute float a_Val1;
attribute float a_Epoch2;
attribute float a_Val2;
attribute float a_color;
</code>    
So we want data formated as x,y,epoch1,val1,epoch2,val2,color.
    <b>x</b> and <b>y</b> will be lng and lat values transformed into pixel coords.
    <b>epoch1</b> and <b>epoch2</b> will be the start and end times for our bubbles.
    <b>val1</b> and <b>val2</b> will be the sizes of the bubble we want to draw.
    <b>color</b> will be a float created by packing a desired RGB value.
    
</p>   

In [ ]:
from PIL import ImageColor
import array, math, os, time
def pack_color(color):
    return (color[0]) + (color[1]) * 256.0 + (color[2]) * 256.0 * 256.0;

def datestr_to_epoch(date_str, format_str):
    return time.mktime(time.strptime(date_str, format_str))

END_EPOCH = format_datestr('2022-12-31', '%Y-%m-%d')

def list_sort(sub_li, idx=0, reverse=False):  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    sub_li.sort(key = lambda x: x[idx], reverse=reverse)
    return sub_li

def topixel(lng, lat):
    lng = float(lng) 
    lat = float(lat)
    x = (lng + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]


cols = "Enabled	Share link identifier	Category	Name	Credits	Base layer	Custom slider ticks	Start date	End date	Step	URL	Scaling	Color Scaling	Map Type	Color	External GeoJSON	Name Key	Show Graph	Graph Title	Graph Plot Colors	Graph Plots First Visible	Graph X-Axis Label	Graph Y-Axis Label	Graph Y-Axis Min	Graph Y-Axis Max	Graph X-Axis Label Interval	Legend Content	Legend Key	Load Data Function	Set Data Function	Number of Levels	Number of Attributes	Vertex Shader	Fragment Shader	Draw Function	Playback Rate	Master Playback Rate	Colormap Src	Layer Description	Featured Themes	Draw Options	Set Data Options	Extras Options	Draw Order	Timeline Type	Layer Constraints	Mapbox	Draw Layer Function"
headers = cols.split("\t")

def print_row(values):
    row = []
    for key in headers:
        if key in values:
            row.append(values[key])
        else:
            row.append("")
    print("\t".join(row))


In [ ]:
densities = list(df['Density Class Range'].unique())
densities = ['0-0.0005', '0.0005-0.005', '0.005-1', '1-10', '>10']
hexes = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
sizes = [2, 4, 8, 16, 32]
points = []
for index, row in df.iterrows():
    x,y = topixel(row['Longitude'], row['Latitude'])
    epoch1 = datestr_to_epoch(row['Date'], '%m/%d/%Y')
    val1 = sizes[densities.index(row['Density Class Range'])]
    epoch2 = END_EPOCH
    val2 = sizes[densities.index(row['Density Class Range'])]
    color = pack_color(ImageColor.getrgb(hexes[densities.index(row['Density Class Range'])]))
    points.append([x,y,epoch1,val1,epoch2,val2,color])
points = list_sort(points, 3, True)
flat_list = []
for row in points:
    flat_list += row
odir = "data/making-earthtime-maps/"
if not os.path.exists(odir):
    os.makedirs(odir)
oname = os.path.join(odir,'bin_bubble_map.bin')
array.array('f', flat_list).tofile(open(oname, 'wb'))
!rsync -rcav data/making-earthtime-maps timemachine2:/t/earthtime.org/app/data

In [ ]:
layer_description = """
 NOAA/ National Centers for Environmental Information (NCEI) (2022): Microplastics Database . [Entire DB]. NOAA National Centers for Environmental Information. Accessed 2022-11-24.
"""
draw_options = """
{
}
"""
fragment_shader = 'WebGLVectorTile2.bubbleMapWithPackedColorFragmentShader'
vertex_shader = 'WebGLVectorTile2.bubbleMapWithPackedColorVertexShader'

values = {
    "Enabled": "TRUE",
    "Share link identifier": "global_marine_microplastics_database_1972_to_present",
    "Category": "Pollution",
    "Name": "Global Marine Microplastics Database 1972 to Present",
    "Credits": "NOAA/NCEI",
    "Base layer": "bdrk",
    "Start date": "1972",
    "End date": "2022",
    "URL": "https://tiles.earthtime.org/making-earthtime-maps/bin_bubble_map.bin",
    "Map Type": "bubble",
    'Legend Content': 'auto',
    'Fragment Shader': fragment_shader,
    'Vertex Shader': vertex_shader,
    'Draw Options': draw_options.replace("\n", ""),
    "Layer Description": layer_description.replace("\n", ""),
    "Load Data Function":"WebGLVectorTile2.prototype._loadData",
    "Set Data Function": "WebGLVectorTile2.prototype._setBufferData",
    "Number of Attributes": "7"}

print_row(values)

In [ ]:
densities = list(df['Density Class Range'].unique())
densities = ['0-0.0005', '0.0005-0.005', '0.005-1', '1-10', '>10']
hexes = ['#440154', '#3b528b', '#21918c', '#5ec962', '#fde725']
sizes = [1, 2, 4, 6, 8]

points = []
for index, row in df.iterrows():
    x,y = topixel(row['Longitude'], row['Latitude'])
    epoch1 = datestr_to_epoch(row['Date'], '%m/%d/%Y')
    val1 = sizes[densities.index(row['Density Class Range'])]
    epoch2 = END_EPOCH
    val2 = sizes[densities.index(row['Density Class Range'])]
    color = pack_color(ImageColor.getrgb(hexes[densities.index(row['Density Class Range'])]))
    points.append([x,y,epoch1,val1,epoch2,val2,color])
points = list_sort(points, 3, True)
flat_list = []
for row in points:
    flat_list += row
odir = "data/making-earthtime-maps/"
if not os.path.exists(odir):
    os.makedirs(odir)
oname = os.path.join(odir,'bin_bubble_map_2.bin')
array.array('f', flat_list).tofile(open(oname, 'wb'))
!rsync -rcav data/making-earthtime-maps timemachine2:/t/earthtime.org/app/data

In [ ]:
layer_description = """
 NOAA/ National Centers for Environmental Information (NCEI) (2022): Microplastics Database . [Entire DB]. NOAA National Centers for Environmental Information. Accessed 2022-11-24.
"""
draw_options = """
{
}
"""
fragment_shader = 'WebGLVectorTile2.bubbleMapWithPackedColorFragmentShader'
vertex_shader = 'WebGLVectorTile2.bubbleMapWithPackedColorVertexShader'

values = {
    "Enabled": "TRUE",
    "Share link identifier": "global_marine_microplastics_database_1972_to_present_2",
    "Category": "Pollution",
    "Name": "Global Marine Microplastics Database 1972 to Present",
    "Credits": "NOAA/NCEI",
    "Base layer": "bdrk",
    "Start date": "2022",
    "End date": "2022",
    "URL": "https://tiles.earthtime.org/making-earthtime-maps/bin_bubble_map_2.bin",
    "Map Type": "bubble",
    'Legend Content': 'auto',
    'Fragment Shader': fragment_shader,
    'Vertex Shader': vertex_shader,
    'Draw Options': draw_options.replace("\n", ""),
    "Layer Description": layer_description.replace("\n", ""),
    "Load Data Function":"WebGLVectorTile2.prototype._loadData",
    "Set Data Function": "WebGLVectorTile2.prototype._setBufferData",
    "Number of Attributes": "7",
    "Draw Options": "{'bubbleAlpha':1.0}"
}

print_row(values)